## 1. THU THẬP DỮ LIỆU

### Import thư viện

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import requests
import re
import os

### Thiết lập trình duyệt

In [2]:
options = Options()
options.add_argument("--headless") # Chạy không hiển thị trình duyệt
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

### Khởi tạo trình duyệt

In [3]:
service = Service("../data/metadata/chromedriver.exe")  # Tải trình duyệt ChromeDriver tương ứng (ở đây tôi sử dụng ChromeDriver 132.0.6834.159)
driver = webdriver.Chrome(service=service, options=options)

In [4]:
save_dir = "../data/raw_images"
os.makedirs(save_dir, exist_ok=True)

In [5]:
def sport_crawler(sport_names, min_images=30, max_images=40):
    failed_sports = []
    
    for sport_name in sport_names:
        keywords = [
            "action real life shot",
            "sports match in progress",
            "iconic sports photo",
            "training session"
        ]
        
        downloaded_images = 0
        for keyword in keywords:
            sport_name_find = f"{sport_name} {keyword}"
            url_sport_name = sport_name_find.replace(" ", "%20")
            url = f"https://www.pinterest.com/search/pins/?q={url_sport_name}"

            try:
                # Mở trang web
                driver.get(url)
                time.sleep(5)  # Chờ trang tải
                
                # Lấy danh sách ảnh
                images = driver.find_elements(By.TAG_NAME, "img")

                for img in images:
                    if downloaded_images >= max_images:
                        break
                    
                    # Lọc ảnh 
                    try:
                        width = int(img.get_attribute("width") or 0)
                        height = int(img.get_attribute("height") or 0)
                        if width < 150 or height < 150:
                            continue  # Bỏ qua ảnh nhỏ
                    except:
                        continue  # Nếu không lấy được kích thước, bỏ qua

                    # Lấy ảnh từ src hoặc srcset
                    img_url = img.get_attribute("srcset") or img.get_attribute("src")
                    if img_url:
                        img_url = img_url.split(",")[-1].split(" ")[0]  # Lấy ảnh lớn nhất trong srcset
                        if not (".jpg" in img_url or ".png" in img_url):
                            continue  # Bỏ qua ảnh không hợp lệ

                        ext = "jpg" if "jpg" in img_url else "png"
                        img_name = f"{sport_name}_{downloaded_images + 1}.{ext}"
                        img_path = os.path.join(save_dir, img_name)

                        try:
                            img_data = requests.get(img_url).content
                            with open(img_path, "wb") as f:
                                f.write(img_data)
                            print(f"✅ Đã tải: {img_path} ({width}x{height})")
                            downloaded_images += 1
                        except Exception as e:
                            print(f"⚠️ Lỗi khi tải ảnh {img_url}: {str(e)}")

            except Exception as e:
                print(f"⚠️ Lỗi khi xử lý môn {sport_name} với từ khóa '{keyword}': {str(e)}")
        
        if downloaded_images < min_images:
            print(f"⚠️ Không đủ ảnh cho môn {sport_name}. Đã tải được {downloaded_images}/{min_images} ảnh.")
            failed_sports.append(sport_name)
        else:
            print(f"🎉 Đã tải đủ {min_images} ảnh cho môn {sport_name}.")

    driver.quit()
    print("🎉 Hoàn thành!")
    return failed_sports

### Main

In [6]:
with open("../data/metadata/sports_cate.txt", "r", encoding="utf-8") as file:
    sports_list = [line.strip() for line in file.readlines()]

# Liệt kê thử vài môn thể thao
sports_list[:5]

['Soccer', 'Basketball', 'Tennis', 'Athletics', 'Swimming']

In [7]:
failed_sports = sport_crawler(sports_list) # Lưu lại danh sách môn thể thao không tải được (đồng thời chạy crawl ảnh tự động)

✅ Đã tải: ../data/raw_images\Soccer_1.jpg (235x353)
✅ Đã tải: ../data/raw_images\Soccer_2.jpg (235x390)
✅ Đã tải: ../data/raw_images\Soccer_3.jpg (235x188)
✅ Đã tải: ../data/raw_images\Soccer_4.jpg (235x353)
✅ Đã tải: ../data/raw_images\Soccer_5.jpg (235x294)
✅ Đã tải: ../data/raw_images\Soccer_6.jpg (235x353)
✅ Đã tải: ../data/raw_images\Soccer_7.jpg (235x294)
✅ Đã tải: ../data/raw_images\Soccer_8.jpg (235x353)
✅ Đã tải: ../data/raw_images\Soccer_9.jpg (235x294)
✅ Đã tải: ../data/raw_images\Soccer_10.jpg (235x332)
✅ Đã tải: ../data/raw_images\Soccer_11.jpg (235x313)
✅ Đã tải: ../data/raw_images\Soccer_12.jpg (235x313)
✅ Đã tải: ../data/raw_images\Soccer_13.jpg (235x235)
✅ Đã tải: ../data/raw_images\Soccer_14.jpg (235x350)
✅ Đã tải: ../data/raw_images\Soccer_15.jpg (235x156)
✅ Đã tải: ../data/raw_images\Soccer_16.jpg (235x156)
✅ Đã tải: ../data/raw_images\Soccer_17.jpg (235x176)
✅ Đã tải: ../data/raw_images\Soccer_18.jpg (235x308)
✅ Đã tải: ../data/raw_images\Soccer_19.jpg (235x356)
✅ 

In [8]:
if failed_sports:
    print(f"🚨 Các môn thể thao không tải được: {', '.join(failed_sports)}")
else:
    print("🎉 Tất cả môn thể thao đã được tải ảnh thành công!")

🎉 Tất cả môn thể thao đã được tải ảnh thành công!
